# Import

In [1]:
import mlflow
import mlflow.keras
import mlflow.tensorflow

import numpy as np
import tensorflow
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

# Setup

In [2]:
mlflow.set_tracking_uri('http://localhost:5000')

class MlFlowModelCheckpoint(tensorflow.keras.callbacks.ModelCheckpoint):
    def __init__(self, artifact_name, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.artifact_name = artifact_name

    def _save_model(self, epoch, logs):
        super()._save_model(epoch, logs)
        mlflow.keras.log_model(self.model, self.artifact_name)

model_checkpoint_callback = MlFlowModelCheckpoint(
    artifact_name='best_model',
    filepath='/data/checkpoints/best_model.h5',
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

params = dict(
    batch_size = 128,
    num_classes = 10,
    epochs = 200,
)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = tf.keras.utils.to_categorical(y_train, params['num_classes'])
y_test = tf.keras.utils.to_categorical(y_test, params['num_classes'])

11490434/11490434 [==============================] - 2s 0us/step


# Run

In [3]:
def make_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(params['num_classes'], activation='softmax'))
    return model

with mlflow.start_run(run_name='mnist_keras_tutorial') as run:
    mlflow.tensorflow.autolog(
        every_n_iter=1
    )
    
    model = make_model()
    model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(
                learning_rate=0.00001,
              ),
              metrics=['accuracy'])
    
    history = model.fit(
        x_train,
        y_train,
        batch_size=params['batch_size'],
        epochs=params['epochs'],
        verbose=1,
        validation_data=(x_test, y_test),
        callbacks=[model_checkpoint_callback],
    )

2022-05-29 21:51:15.319084: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/200
467/469 [============================>.] - ETA: 0s - loss: 2.3343 - accuracy: 0.0993

TypeError: _save_model() got an unexpected keyword argument 'batch'